In [1]:
import os

SEASONS = list(range(2016,2023))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [2]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [5]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [6]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [7]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [10]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Refere

In [11]:
standings_files = os.listdir(STANDINGS_DIR)

In [12]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [13]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

76ers vs Hornets, April 1, 2016 | Basketball-Reference.com
Mavericks vs Pistons, April 1, 2016 | Basketball-Reference.com
Nets vs Knicks, April 1, 2016 | Basketball-Reference.com
Cavaliers vs Hawks, April 1, 2016 | Basketball-Reference.com
Raptors vs Grizzlies, April 1, 2016 | Basketball-Reference.com
Magic vs Bucks, April 1, 2016 | Basketball-Reference.com
Timberwolves vs Jazz, April 1, 2016 | Basketball-Reference.com
Heat vs Kings, April 1, 2016 | Basketball-Reference.com
Celtics vs Warriors, April 1, 2016 | Basketball-Reference.com
Wizards vs Suns, April 1, 2016 | Basketball-Reference.com
Pacers vs 76ers, April 2, 2016 | Basketball-Reference.com
Pistons vs Bulls, April 2, 2016 | Basketball-Reference.com
Raptors vs Spurs, April 2, 2016 | Basketball-Reference.com
Kings vs Nuggets, April 2, 2016 | Basketball-Reference.com
Heat vs Trail Blazers, April 2, 2016 | Basketball-Reference.com
Pelicans vs Nets, April 3, 2016 | Basketball-Reference.com
Hornets vs Cavaliers, April 3, 2016 | Baske

Error: Page.goto: net::ERR_NAME_NOT_RESOLVED at https://www.basketball-reference.com/boxscores/201704220ATL.html
Call log:
navigating to "https://www.basketball-reference.com/boxscores/201704220ATL.html", waiting until "load"


In [5]:
import requests
import random
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
import os


In [7]:

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}
def scrape_stat_per_game(headers):
    """Scrapes NBA player salary data from Basketball Reference."""
    url = "https://www.basketball-reference.com/contracts/players.html"
    
    # Make the HTTP request
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        return

    # Introduce a random delay to avoid detection
    wait_time = random.uniform(3, 5)
    time.sleep(wait_time)
    print(f"Waited for {wait_time:.2f} seconds before parsing the page.")

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table with the specific ID
    stat_table = soup.find("table", id="player-contracts")
    if stat_table is None:
        print("Error: Could not find the table with ID 'player-contracts'")
        return
    
    # Extract table headers
    headers = [th.getText() for th in stat_table.find_all('th')][1:]  # Skip the first empty header

    # Extract table rows
    rows = stat_table.find_all('tr')
    data = []
    for row in rows:
        cells = row.find_all('td')
        if cells:
            data.append([cell.getText().strip() for cell in cells])

    # Save to CSV
    output_dir = "players_salaries"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_file = os.path.join(output_dir, "NBA_players_salaries.csv")
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write headers
        writer.writerows(data)   # Write rows

    print(f"Data successfully saved to {output_file}")

# Run the scraper
scrape_stat_per_game(headers=headers)

Waited for 4.22 seconds before parsing the page.
Data successfully saved to players_salaries\NBA_players_salaries.csv
